In [3]:

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import cupy as cp
import json

# Открытие файлов
train_path = "./data/train"  # Замените на путь к вашему train файлу
test_no_reference_path = "./data/test_no_reference"  # Замените на путь к вашему test файлу

# Загрузка данных
with open(train_path, 'r', encoding='utf-8') as train_file:
    train_data = [json.loads(line) for line in train_file.readlines()]

with open(test_no_reference_path, 'r', encoding='utf-8') as test_file:
    test_data = [json.loads(line) for line in test_file.readlines()]

# Преобразование данных в DataFrame
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

# Векторизация с использованием TF-IDF
vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(2, 4))
train_src_vectors = vectorizer.fit_transform(train_df['src'])
test_src_vectors = vectorizer.transform(test_df['src'])

# Перенос на GPU (если возможно)
try:
    train_src_vectors = cp.sparse.csr_matrix(train_src_vectors.get())
    test_src_vectors = cp.sparse.csr_matrix(test_src_vectors.get())
    use_gpu = True
    print("GPU будет использоваться для расчетов.")
except:
    use_gpu = False
    print("GPU недоступен. Используются CPU-вычисления.")

# Расчет косинусного сходства и получение лучших совпадений
results = []
for i in tqdm(range(test_src_vectors.shape[0]), desc="Matching", unit="row"):
    if use_gpu:
        similarities = train_src_vectors.dot(test_src_vectors[i].T).toarray()
        best_match_index = cp.argmax(similarities).get()
    else:
        similarities = cosine_similarity(test_src_vectors[i], train_src_vectors)
        best_match_index = similarities.argmax()
    results.append(train_df.iloc[best_match_index]['dst'])

# Добавление результатов в тестовый DataFrame
test_df['dst'] = results

# Сохранение в формате JSON Lines
output_path = "./data/test_with_translations.jsonl"  # Укажите путь для сохранения результата
with open(output_path, 'w', encoding='utf-8') as output_file:
    for _, row in test_df.iterrows():
        json.dump(row.to_dict(), output_file, ensure_ascii=False)
        output_file.write('\n')

print(f"Результаты сохранены в {output_path}.")


GPU недоступен. Используются CPU-вычисления.


Matching: 100%|████████████████████████████████████████████████████████████| 1000/1000 [15:36<00:00,  1.07row/s]

Результаты сохранены в ./data/test_with_translations.jsonl.


In [ ]:
0.85